In [18]:
import re
import codecs
import glob
import csv
import pandas as pd
import numpy as np
import pickle
%autosave 5

Autosaving every 5 seconds


# Constructing Standard Data Files for Online Participants
##### 5/27/20
#### i. Choose output name

In [34]:
pickle_name = "pickle/data_dict_pilots_b1_b2_b3_b4_b5_b6.p"
#pickle_name = "pickle/data_dict_b6.p"

## I. Treating Log Files
### A. Reading in Log Files

In [35]:
#pilot1
files1 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/*.log")

#pilot2
files2 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot2/8/*.log")

#pilot3
files3 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot3/4/*.log")

#pilot4
files4 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot4/7/*.log")

#final ver- p
filesp = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_p/10/*.log")

#final ver- b1
filesb1 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/*.log")

#final ver- b2
filesb2 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/*.log")

#final ver- b3
filesb3 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/*.log")

#final ver- b4
filesb4 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b4/29/*.log")

#final ver - b5
filesb5 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/*.log")

#final ver - b6
filesb6 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/*.log")




### (Adjust here which groups to include in analysis)

In [38]:
files = files1 + files2 + files3 + files4 + filesp + filesb1 + filesb2 + filesb3 + filesb4 + filesb5 + filesb6
#files = filesb6
# we'll use a regex for this even though they're ugly and painful:
scanQuote = re.compile(r"'(.*?)'")  # finds any number of words within a quote
print(len(files))

327


### B. Import Qualtrics CSV

In [39]:
q_df = pd.read_csv("qualtrics/qualtrics.csv")

### C. Import CSVs

In [40]:
#FullForce Behavioral

#pilot1
csv1 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/*.csv")

#pilot2
csv2 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot2/8/*.csv")

#pilot3
csv3 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot3/4/*.csv")

#pilot4
csv4 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot4/7/*.csv")

#final ver- p
csvp = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_p/10/*.csv")

#final ver- b1
csvb1 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/*.csv")

#final ver- b2
csvb2 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/*.csv")

#final ver- b3
csvb3 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/*.csv")

#final ver- b4
csvb4 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b4/29/*.csv")

#final ver- b5
csvb5 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/*.csv")

#final ver- b6
csvb6 = glob.glob("/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/*.csv")

### (Insert here which files to include)

In [41]:
csvnames = csv1 + csv2 + csv3 + csv4 + csvp + csvb1 + csvb2 + csvb3 + csvb4 + csvb5 + csvb6
#csvnames = csvb6

csv_df = {}

for csvname in csvnames:
    #print(csvname)
    label = csvname[:-4]
    
    csv_df[label] = pd.read_csv(csvname)
    
print('done!')

done!


### D. Creating dictionary files_pd with log files data filtered (key: filename .log)

In [42]:
#instantiate dictionary
files_pd = {}

for filename in files:
    #print(filename)
    # open log file
    thisFile = codecs.open( filename, 'r', 'utf-8')
    #initiate a dataframe
    df = pd.DataFrame(columns=['time','keypress'])
    #go through each line in the file
    for line in thisFile.readlines():
        if "DATA" in line:  # new trial reset all entries
            if "Keydown:" in line:
                first, keypress = line.split("Keydown:")
                time = first.split("\t")[0]
                keypress = keypress.strip()
                df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
            else:
                pass
        if "text_34: autoDraw = false" in line:
            time = "here"
            keypress = 33
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        #recording the distractor words    
        if "text_66: text =" in line:
            time = 'distractor_text'
            b, keypress = line.split("text = ")
            keypress = keypress.strip()
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        #mark the end of the distractor typing
        if "instr_9: autoDraw = false" in line:
            time = 44
            keypress = 0
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        #recording the short answer questions
        if "instr: text =" in line:
            b, keypress = line.split("text = ")
            time = 'Q1'
            keypress = keypress.strip()
            if keypress == "\"default text\"":
                pass
            else:
                df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "instr_2: text =" in line:
            b, keypress = line.split("text = ")
            time = 'Q2'
            keypress = keypress.strip()
            if keypress == "\"default text\"":
                pass
            else:
                df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "instr_3: text =" in line:
            b, keypress = line.split("text = ")
            time = 'Q3'
            keypress = keypress.strip()
            if keypress == "\"default text\"":
                pass
            else:
                df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "instr_4: text =" in line:
            b, keypress = line.split("text = ")
            time = 'Q4'
            keypress = keypress.strip()
            if keypress == "\"default text\"":
                pass
            else:
                df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "instr_5: text =" in line:
            b, keypress = line.split("text = ")
            time = 'Q5'
            keypress = keypress.strip()
            if keypress == "\"default text\"":
                pass
            else:
                df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "instr_6: text =" in line:
            b, keypress = line.split("text = ")
            time = 'Q6'
            keypress = keypress.strip()
            if keypress == "\"default text\"":
                pass
            else:
                df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "instr_7: text =" in line:
            b, keypress = line.split("text = ")
            time = 'Q7'
            keypress = keypress.strip()
            if keypress == "\"default text\"":
                pass
            else:
                df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "instr_8: text =" in line:
            b, keypress = line.split("text = ")
            time = 'Q8'
            keypress = keypress.strip()
            if keypress == "\"default text\"":
                pass
            else:
                df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        #short answers
        text_num = ["43", "44", "45", "46", "47", "48", "51", "53"]
        if "text_43: text = " in line:
            b, keypress = line.split("text = ")
            time = 'A1'
            keypress = keypress.strip()
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "text_44: text = " in line:
            b, keypress = line.split("text = ")
            time = 'A2'
            keypress = keypress.strip()
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "text_45: text = " in line:
            b, keypress = line.split("text = ")
            time = 'A3'
            keypress = keypress.strip()
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "text_46: text = " in line:
            b, keypress = line.split("text = ")
            time = 'A4'
            keypress = keypress.strip()
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "text_47: text = " in line:
            b, keypress = line.split("text = ")
            time = 'A5'
            keypress = keypress.strip()
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "text_48: text = " in line:
            b, keypress = line.split("text = ")
            time = 'A6'
            keypress = keypress.strip()
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "text_51: text = " in line:
            b, keypress = line.split("text = ")
            time = 'A7'
            keypress = keypress.strip()
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
        if "text_53: text = " in line:
            b, keypress = line.split("text = ")
            time = 'A8'
            keypress = keypress.strip()
            df = df.append({'time' : time , 'keypress' : keypress} , ignore_index=True)
    #remove rows of pd with empty keypress values
    df = df[df.keypress != '""']
    df = df[df.keypress != '']
    #reset index
    df = df.reset_index(drop=True)
    #add the df to the dictionary
    files_pd[filename] = df
    

        

## II. Building Standardized Data File 
#### pd w cols = story, type, persp, alt_persp, sa_order, story_presses, distractors, 'SA's

## A. Instantiate df data_dict (key: filename)

In [43]:
data_dict = {}

#fill data_dict with all files
for filename in files:
    this_df = pd.DataFrame()
    label = filename[:-4]
    data_dict[label] = this_df
    

In [44]:
len(csvnames)

327

## C. Extract information from files_pd
### i. story_presses

In [45]:
#include if statement, since if the values are in the .csv we will use those, maybe not actually. If this method works
#why not make it ubiquitous for all. I will do some comparisons with people who do have story_presses in their .csv
#to verify this extraction method

for filename in files:
    print(filename)
    #call the dataframe for this file
    df = files_pd[filename]
    #finding index of 'here'
    here = np.array(df.index[df['time'] == 'here'].tolist())
    #subtracting 1 to get index of keypresses
    seg = here - 1
    #getting the keypress values
    key_vals = df['keypress'][seg].tolist()
    # Removing 'F1' from subj finalver_b5/78/149_1
    if filename == '/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/149_1_online_exp_2020-09-28_15h51.38.989.log':
        key_vals = ['1' if x == 'F1' else x for x in key_vals]
    #turning keypresses into ints
    key_vals = [int(i) for i in key_vals]
    #adjusting the filename to match the key
    label = filename[:-4]
    #adding the key_vals to the df
    data_dict[label]['story_presses'] = key_vals

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-06_20h00.18.643.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-07_00h46.51.319.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-07_23h36.54.796.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-07_23h40.02.268.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h01.48.508.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h05.05.369.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h54.27.962.log
/Users/alexreblando/

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/361_1_online_exp_2020-07-16_04h07.12.067.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/362_1_online_exp_2020-07-16_09h07.16.604.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/366_1_online_exp_2020-07-16_09h06.28.811.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/375_1_online_exp_2020-07-16_09h07.27.451.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/376_1_online_exp_2020-07-16_09h08.08.638.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/378_1_online_exp_2020-07-16_09h08.18.318.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/379_1_online_exp_2020-07-16_09h07.20.528.log
/Users/alexreblando/Documents/GitH

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/64_1_online_exp_2020-09-28_14h32.29.082.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/66_1_online_exp_2020-09-28_15h33.23.105.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/68_1_online_exp_2020-09-28_15h32.44.170.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/69_1_online_exp_2020-09-28_13h10.55.410.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/71_1_online_exp_2020-09-28_14h32.35.186.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/74_1_online_exp_2020-09-28_14h32.33.692.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/75_1_online_exp_2020-09-28_15h32.26.227.log
/Users/alexreblando/Documents/GitHub/data

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/6_1_online_exp_2020-10-20_10h04.26.275.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/71_1_online_exp_2020-10-20_12h41.20.987.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/76_1_online_exp_2020-10-20_10h15.21.840.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/77_1_online_exp_2020-10-20_12h45.19.440.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/84_1_online_exp_2020-10-20_13h00.42.305.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/85_1_online_exp_2020-10-21_00h21.05.606.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/8_1_online_exp_2020-10-20_10h07.07.928.log
/Users/alexreblando/Documents/GitHub/data_a

### ii. short answers

In [46]:
Alist = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8']

for filename in files:
    df = files_pd[filename]
    for thisA in Alist:
        #finding index of thisA
        here = np.array(df.index[df['time'] == thisA].tolist())
        #getting the keypress values
        key_vals = df['keypress'][here].tolist()
        if not key_vals:
            this_val = ''
        else:
            #index the last value
            this_val = key_vals[-1]
        label = filename[:-4]
        #adding the key_vals to the df
        data_dict[label][thisA] = this_val

### iii. distractors

In [47]:
for filename in files:
    print(filename)
    #get the label for the data_dict
    label = filename[:-4]
    #select df from files_pd
    df = files_pd[filename]
    #finding the index of 44, the marker for distractor texts
    here = np.array(df.index[df['time'] == 44].tolist())
    #subtracting 2 to get index of keypresses
    distractors = here - 2
    distractor_vals = df['keypress'][distractors].tolist()
    for i in range(0, 5):
        data_dict[label]['distractor' + str(i+1)] = distractor_vals[i]

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-06_20h00.18.643.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-07_00h46.51.319.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-07_23h36.54.796.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-07_23h40.02.268.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h01.48.508.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h05.05.369.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h54.27.962.log
/Users/alexreblando/

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/182_1_online_exp_2020-06-24_15h29.38.130.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/183_1_online_exp_2020-06-24_15h35.15.415.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/186_1_online_exp_2020-06-24_10h34.36.768.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/187_1_online_exp_2020-06-24_09h35.40.463.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/189_1_online_exp_2020-06-24_10h34.33.295.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/192_1_online_exp_2020-06-24_15h34.11.606.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/195_1_online_exp_2020-06-24_09h34.43.267.log
/Users/alexreblando/Documents/GitH

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b4/29/487_1_online_exp_2020-07-29_19h19.53.250.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b4/29/488_1_online_exp_2020-07-29_14h20.01.678.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b4/29/489_1_online_exp_2020-07-29_15h20.23.530.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b4/29/490_1_online_exp_2020-07-29_19h19.53.518.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b4/29/493_1_online_exp_2020-07-29_14h21.18.641.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b4/29/494_1_online_exp_2020-07-29_19h20.45.344.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b4/29/495_1_online_exp_2020-07-29_13h22.57.572.log
/Users/alexreblando/Documents/GitH

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/148_1_online_exp_2020-10-20_13h50.10.219.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/149_1_online_exp_2020-10-20_14h50.03.916.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/151_1_online_exp_2020-10-20_15h55.38.330.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/152_1_online_exp_2020-10-20_14h50.23.352.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/153_1_online_exp_2020-10-20_15h11.14.280.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/154_1_online_exp_2020-10-20_11h54.23.748.log
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/156_1_online_exp_2020-10-20_13h56.22.687.log
/Users/alexreblando/Documents/GitH

IndexError: list index out of range

## D. Extract info from CSV files
### i. Get prolific_id, story, type, persp, alt_persp, sa_order

In [48]:
for filename in files:
    #get the label for the data_dict
    label = filename[:-4]
    #getting prolific id
    data_dict[label]['prolific_id'] = csv_df[label]['Prolific_ID'].iloc[0]
    #getting story number
    data_dict[label]['story'] = int(csv_df[label]['story'].iloc[0])
    #getting priming type
    data_dict[label]['p_type'] = int(csv_df[label]['val2'].iloc[0])
    #perspective
    if "persp_val" in csv_df[label]:
        data_dict[label]['persp'] = int(csv_df[label]['persp_val'].iloc[0])
        #alternative perspective
        data_dict[label]['alt_persp'] = int(csv_df[label]['alt_persp_val'].iloc[0])
    else:
        data_dict[label]['persp'] = int(csv_df[label]['perp_val'].iloc[0])
        #alternative perspective
        data_dict[label]['alt_persp'] = int(csv_df[label]['alt_perp_val'].iloc[0])
    #short answer order
    if "sa_order" in csv_df[label]:
        data_dict[label]['sa_order'] = int(csv_df[label]['sa_order'].iloc[0])
    #if the short answer order is not in the file (pilot i, ii) then put the first sa question in 'sa_order' column    
    else:
        data_dict[label]['sa_order'] = files_pd[filename].loc[files_pd[filename]['time'] == 'Q1', 'keypress'].iloc[0]
        
        
    
    

## E. Recall Data from Qualtrics

### ii. Insert Recall Data from Qualtrics CSV into data_dict using prolific_id

In [49]:
for key in data_dict.keys():
    #print(key)
    #getting prolific id from data_dict
    this_id = data_dict[key]['prolific_id'].iloc[0]
    print(this_id)
    this_id_new = this_id.strip()
    print(this_id)
    if this_id in q_df.values:
        #index of row with this_id
        data_dict[key]['recall'] = q_df.loc[q_df['Q1'] == this_id, 'Q8'].iloc[0]
    elif this_id_new in q_df.values:
        data_dict[key]['recall'] = q_df.loc[q_df['Q1'] == this_id_new, 'Q8'].iloc[0]
  

5ea07ccfcc5aed04bc7e4148
5ea07ccfcc5aed04bc7e4148
5d80f95b43119f0015986eb1
5d80f95b43119f0015986eb1
5c4271f51b4bf300018b0920
5c4271f51b4bf300018b0920
5c1cfb57e2d9150001b5975b
5c1cfb57e2d9150001b5975b
5e84ab6c1c387d0424a1b662
5e84ab6c1c387d0424a1b662
5d0e2a06facd7500197a46e3
5d0e2a06facd7500197a46e3
5cdeaeb3557fbd00018a2c59
5cdeaeb3557fbd00018a2c59
5cf0a300b094d700183e4d54
5cf0a300b094d700183e4d54
5ce5b9844cd8130019acba32
5ce5b9844cd8130019acba32
597cb71cd4e83700016c35b2
597cb71cd4e83700016c35b2
5e7b804f9379140478c688da
5e7b804f9379140478c688da
5eac5376dfc1010c08a633bb
5eac5376dfc1010c08a633bb
5cea26087afd86001a34a5db
5cea26087afd86001a34a5db
5deae08fe4f84020dfafac72
5deae08fe4f84020dfafac72
5bc9e924e642db0001d3a1c5
5bc9e924e642db0001d3a1c5
5cc482bd8d2a1e001885c450
5cc482bd8d2a1e001885c450
5d82994758861000188f76dc
5d82994758861000188f76dc
58af6fa166d1330001eebaab
58af6fa166d1330001eebaab
5e3f032709a1141608164346
5e3f032709a1141608164346
5d02c0e382e4f00001aa20cd
5d02c0e382e4f00001aa20cd


5f325ad3c8616a1b5f8c7bd4
5f325ad3c8616a1b5f8c7bd4
5f6d646d819cf31de8f64a78
5f6d646d819cf31de8f64a78
5f514f60277f9b3787539994
5f514f60277f9b3787539994
5f5b550f46c58b26f4ce4397
5f5b550f46c58b26f4ce4397
5c3341bc21f99c000175c5cc
5c3341bc21f99c000175c5cc
5f4a68e4d8963d5e469541f1
5f4a68e4d8963d5e469541f1
 5e83abb4e0a2e40fd565226d
 5e83abb4e0a2e40fd565226d
5f4938236f4ed910789a1307
5f4938236f4ed910789a1307
56b4b565b2de2a000d3316ba
56b4b565b2de2a000d3316ba
5f52afc954863a5bd06a1665
5f52afc954863a5bd06a1665
559e6830fdf99b4f0b98514a
559e6830fdf99b4f0b98514a
5e8b149e1f49d702dadae9ab
5e8b149e1f49d702dadae9ab
5ecd4acdb218ec085c6eefe6
5ecd4acdb218ec085c6eefe6
5f3aead81965e27c4a531d95
5f3aead81965e27c4a531d95
5ed9c0336cb961557325f482
5ed9c0336cb961557325f482
5f58fdc20fcd710ce090cdc1
5f58fdc20fcd710ce090cdc1
5c2b4b959f18a9000179a141
5c2b4b959f18a9000179a141
5f6c72f5500db10161918ca4
5f6c72f5500db10161918ca4
5f50d96e44327d254d59bce7
5f50d96e44327d254d59bce7
5f56056a645a6e9dba848fa2
5f56056a645a6e9dba848fa

AttributeError: 'numpy.float64' object has no attribute 'strip'

## F. Pickling

In [50]:
pickle.dump( data_dict, open( pickle_name, "wb" ) )